## This notebook demonstrates the use of a simple meal planner expert system using Canada's Food and Nutrition database as the Knowledge Base

In [1]:
#import libraries
import numpy as np
import pandas as pd
import random
from IPython.display import display

In [2]:
class UserData():
    """This class stores and calculates user specific data"""
    def __init__(self):
        """Initialize User Data Variables"""
        self.age = None
        self.height = None
        self.gender = None
        self.weight = None
        self.prot_req = None
        self.carb_req = None
        self.fat_req = None
        #self.ingredient_df = None
        
    def input_user_data(self,input_list):
        """Use input list to initialize user data"""
        self.age = input_list[0]
        self.height = input_list[1]
        self.gender = input_list[2]
        self.weight = input_list[3]                
        
    def calculate_daily_food_requirements(self):
        """This function uses the Harris-Benedict Equation for Basal Energy Expenditure to 
           to calculate the daily requirement of protein, fat and carbohydrates"""
        if self.gender == 1:
            multipliers = [655.1,9.6,1.9,4.7]
        else:
            multipliers = [66.5,13.8,5.0,6.8]
        #HBE to find BEE
        total_calories = multipliers[0] + multipliers[1] * self.weight\
                        + multipliers[2] * self.height + multipliers[3] * self.age
        self.prot_req = self.weight
        prot_per = ((self.weight * 4)/total_calories) * 100
        carb_per = 60
        fat_per = 100 - (prot_per + carb_per)
        self.carb_req = ((total_calories * carb_per)/100)/4
        self.fat_req = ((total_calories * fat_per)/100)/9
        print("You require",self.prot_req,"g of protein",self.carb_req,"g of carbohydrates",\
             self.fat_req,"g of fats per day")

In [3]:
user = UserData()
user.input_user_data([25,178,0,95]) #[Age,Height,Gender(0 for male 1 for female),weight]
user.calculate_daily_food_requirements()

You require 95 g of protein 365.625 g of carbohydrates 66.11111111111111 g of fats per day


In [37]:
class Meal():
    """Meal class holds the information for one day's meal"""
    
    def __init__(self,ingredients):
        """Creates a Random Meal from User Data OR from specified dataframe"""  
        self.fitness = 0.0
        self.ingredients = ingredients
        self.ingredients['MealAmount'] = 0.0
        self.ingredients['MealAmount'] = self.ingredients['MealAmount'].apply(lambda x: random.random() * 2000)
        
    def get_total_vals(self):
        total_vals = {
            "PROTValue":0.0,
            "FATValue":0.0,
            "CARBValue":0.0,
            "STARValue":0.0,
            "TSUGValue":0.0,
            "TDFValue":0.0,
            "TSATValue":0.0,
            "MUFAValue":0.0,
            "PUFAValue":0.0
        }
        for i,row in self.ingredients.iterrows():
            for column in self.ingredients.columns:
                if column in total_vals:
                    total_vals[column] = total_vals[column] + row.MealAmount * row[column]/100
        return total_vals
               
    def update_fitness(self,prot_req,carb_req,fat_req):
        total_vals = self.get_total_vals()
        #apply rules to calculate fitness
        #add the percentage of protein, carbohydrate and fat value met and subtract if extra
        fitness_multiplier = 1
        if total_vals["PROTValue"] > prot_req:
            fitness_multiplier = 2
        self.fitness = self.fitness + fitness_multiplier * (prot_req - total_vals["PROTValue"])/prot_req
        fitness_multiplier = 1
        if total_vals["CARBValue"] > carb_req:
            fitness_multiplier = 2
        self.fitness = self.fitness + fitness_multiplier * (carb_req - total_vals["CARBValue"])/carb_req
        fitness_multiplier = 1
        if total_vals["FATValue"] > fat_req:
            fitness_multiplier = 2
        self.fitness = self.fitness + fitness_multiplier * (fat_req - total_vals["FATValue"])/fat_req
        
        ing_per = total_vals["TSUGValue"]/total_vals["CARBValue"]
        if ing_per > 0.3: #if sugar is more than 30% of carb value then meal is not good
            self.fitness = self.fitness - ing_per
        ing_per = total_vals["TSATValue"]/total_vals["FATValue"]
        if ing_per > 0.3: #if saturated fat is more than 30% of fat value then meal is not good
            self.fitness = self.fitness - ing_per
        
        #add total dietary fibre percentage  
        #self.fitness = self.fitness + (total_vals["TDFValue"]/total_vals["CARBValue"])       
        #add total unsaturated fat percentage
        #self.fitness = self.fitness + ((total_vals["MUFAValue"] + total_vals["PUFAValue"])/total_vals["FATValue"])
        

class DailyMealPlanner():
    """Implementation of Expert System that gives a new meal plan daily"""
    def __init__(self,
                 user_data,
                 combinations = 100,
                 no_of_ingredients = 5,
                 mutation_rate=0.001,
                 elite_percentage=10,
                 random_state=1):
        """Initialize The Meal Plan System"""
        self.combinations = combinations
        self.no_of_ingredients = no_of_ingredients
        self.mutation_rate = mutation_rate
        self.elite_per_unit = elite_percentage/100.0
        self.random_state = random_state
        self.user_data = user_data
        self.food_data = pd.read_csv('FoodNutritionData.csv')
        self.food_data = self.food_data[~self.food_data.FoodGroup.isin(
            ["Babyfoods","Beverages","Fats and Oils","Spices and Herbs"])].copy()
        self.increment = 0
    
    def create_population(self):
        """Get Random Samples from ingredient data"""
        random.seed(self.random_state)
        self.increment = self.increment + 1
        self.current_generation = [] #create random new meals from user ingredients
        for i in range(0,self.combinations):
            self.current_generation.append(Meal(self.food_data.sample(
                n = self.no_of_ingredients,
                random_state = self.random_state + self.increment)))
            self.increment = self.increment + 1
            
    def update_generation_order(self):
        """Updates the fitness function of the meal and sets the order of the current generation"""
        for meal in self.current_generation:
            meal.update_fitness(self.user_data.prot_req,self.user_data.carb_req,self.user_data.fat_req)
        #rank meals in order of fitness
        self.current_generation = sorted(self.current_generation, key=lambda meal: meal.fitness, reverse=True)
        
    def perform_selection(self):
        """Performs the selection step of genetic algorithm"""
        total_selected = int(self.elite_per_unit * len(self.current_generation))
        selected_generations = []
        for i in range(0,total_selected):
            selected_generations.append(self.current_generation[i])
        # randomly pick candidates from the rest
        selected_generations.extend(random.sample(self.current_generation[int(self.elite_per_unit):], 10))
        self.current_generation = selected_generations
    
    def perform_ordered_cross_over(self):
        """Performs the ordered cross over step of genetic algorithm"""
        #shuffle selected generation
        self.current_generation = random.sample(self.current_generation,len(self.current_generation))
        children = []
        while len(children) < self.combinations:
            x,y = random.sample(range(len(self.current_generation)), 2)
            parent_meal_1 = self.current_generation[x]
            parent_meal_2 = self.current_generation[y]
            child_ingredients = [None] * self.no_of_ingredients #create empty list of ingredients
            p1, p2 = random.sample(range(self.no_of_ingredients), k=2) #get two points in list
            from_index = min(p1, p2) #arrange in min max
            to_index = max(p1, p2)
            parent_1_genes = parent_meal_1.ingredients.to_dict(orient='records')
            child_ingredients[from_index:to_index] = parent_1_genes[from_index:to_index] #copy records from parent 1
            parent_2_genes = parent_meal_2.ingredients.to_dict(orient='records')
            
            for i in range(0,len(child_ingredients)): #fill rest with parent 2 records
                if child_ingredients[i] is None:
                    ingredient_val = None
                    for ingredient in parent_2_genes:
                        if ingredient not in child_ingredients:
                            ingredient_val = ingredient
                            break
                    if ingredient_val is None: #didn't find unique ingredient, get from dataset
                        ingredient_val = self.get_unique_ingredient(child_ingredients)
                    child_ingredients[i] = ingredient_val
            #create child from parent ingredients
            child = Meal(pd.DataFrame(child_ingredients))
            #add to next generation 
            children.append(child)
        self.current_generation = children
        
    def get_unique_ingredient(self,ing_list):
        ingredient_val = None
        while ingredient_val is None:
            ingredient_val = self.food_data.sample(n = 1,random_state = self.random_state + self.increment).to_dict(orient='records')[0]
            self.increment = self.increment + 1
            if ingredient_val in ing_list:
                ingredient_val = None
        return ingredient_val
        
    def perform_mutation(self):
        """Performs the mutation step of genetic algorithm"""
        for meal in self.current_generation:
            #get ingredients of current meal
            ingredients = meal.ingredients.to_dict(orient='records')
            for i in range(self.no_of_ingredients):
                if (random.random() < self.mutation_rate):
                    ingredients[i] = self.get_unique_ingredient(ingredients) #replace with fresh ingredient
            meal.ingredients = pd.DataFrame(ingredients) #set changes to ingredients dataframe
        
    def next_generation(self):
        """Runs a single iteration of genetic algorithm"""
        self.update_generation_order()
        self.perform_selection()
        self.perform_ordered_cross_over()
        self.perform_mutation()
            
    def get_healthy_meal(self,generations = 100):
        """Runs the Genetic Algorithm to get a new meal"""
        self.create_population()
        for i in range(0,generations): #run for generations
            self.next_generation()
        self.update_generation_order()
        return self.current_generation[0] #return best generation

In [38]:
daily_meal_planner = DailyMealPlanner(user,100,10,0.002,10,5)
new_meal = daily_meal_planner.get_healthy_meal(100)
print("Meal Fitness:" ,new_meal.fitness)
display(new_meal.ingredients.head(10))
print("Meal Nutrition:",new_meal.get_total_vals())

Meal Fitness: -0.19283689637503093


,FoodID,FoodDescription,FoodGroup,PROTValue,FATValue,CARBValue,STARValue,TSUGValue,TDFValue,TSATValue,MUFAValue,PUFAValue,MealAmount
0,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,1584.814714
1,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,178.869731
2,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,163.000535
3,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,202.351030
4,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,1457.189113
5,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,602.390865
6,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,1266.562870
7,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,729.732684
8,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,154.205801
9,501618,"Cauliflower, boiled, drained, with salt",Vegetables and Vegetable Products,1.84,0.45,4.11,0.0,1.86,2.3,0.07,0.03,0.21,247.673409


Meal Nutrition: {'PROTValue': 121.19694985273661, 'FATValue': 29.640558387897542, 'CARBValue': 270.7170999427976, 'STARValue': 0.0, 'TSUGValue': 122.51430800330984, 'TDFValue': 151.49618731592076, 'TSATValue': 4.610753527006285, 'MUFAValue': 1.9760372258598358, 'PUFAValue': 13.83226058101885}
